In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense, LSTM
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

# 문장 데이터를 100개로 확장
texts = [
    # 퇴사 발언 (50개)
    "네가 못해서 팀 전체가 고생하는 거야 💻🐛",
    "디버깅도 못 하는 개발자가 개발자라고 할 수 있나? 🤦‍♂️",
    "주말에 배포된 코드, 네가 고쳐야지. 쉬는 날도 네가 책임져 🛠️",
    "회사 서버 터지면 네가 잠 깨서라도 고쳐야지 😴🔧",
    "네가 짠 코드는 장애 유발의 근원 같아 ⚠️",
    "실력 없는 개발자는 그냥 코딩만 하지 말고 문서 작성이나 해 📝",
    "네가 만든 API는 불안정 그 자체야 🌐🚨",
    "테스트 없이 코드를 올렸다고? 넌 진짜 무모하다 💥",
    "야근비? 네가 회사에서 돈을 벌 가치가 있다고 생각해? 💸",
    "팀의 속도를 늦추는 원인이 뭔지 생각해 봐 😡",
    "왜 퇴근 시간이 되었는데도 일이 안 끝나? ⏰",
    "지금 상황에서 버그를 잡아야 하는 건 네 책임이야 🐞",
    "커밋 메시지가 이렇게 불친절해서야 😤",
    "네가 만든 UI는 사용자 경험을 망치고 있어 📉",
    "프로젝트의 발목을 잡는 코드는 네가 항상 짜더라 🤷‍♀️",
    "코드 리뷰 시간 줄이려면 네 코드부터 제대로 짜 🛑",
    "왜 서버 로그가 다 빨간색인지 네가 설명 좀 해 봐 🔴📜",
    "테크 리더가 네 코드를 신뢰하지 못한다는 거 몰라? 🚫",
    "내일까지 기능 다 완성 못 하면 책임은 네 거야 🕛",
    "결과물이 이 정도라니, 실망스러워 😔",
    "우리 회사에 이렇게 실력 없는 사람이 있었다니 🙄",
    "왜 배포 이후 장애가 생겼는지 아직도 모른다고? 😡",
    "왜 코드가 느린지 네가 설명 좀 해 줘 봐 🐌",
    "그 정도 실력으로 개발자는 너무 과분하지 않나? 🤔",
    "이건 네 책임이니까 끝날 때까지 집에 가지 마 🏢",
    "왜 그렇게 기본적인 걸 몰라? 📘",
    "주말에 회사 서버 터졌는데 왜 연락 안 받았어? 📞",
    "배포 전 테스트를 빼먹었다니, 네가 짠 코드는 신뢰할 수 없어 🚧",
    "이 프로젝트가 느린 건 네가 짠 코드 때문이야 💥",
    "왜 다른 팀원이 항상 네 코드를 고쳐야 하지? 🤷‍♂️",
    "네가 만든 함수는 진짜 끔찍해 😨",
    "프로덕션 환경에서 네 코드는 언제나 장애를 유발해 🔥",
    "이 정도 결과물이면 더 이상 할 말이 없다 😞",
    "프로그래밍을 다시 배워야 할 것 같아 🧑‍🎓",
    "이런 식이면 더 좋은 개발자를 찾는 게 낫겠어 🕵️‍♂️",
    "네가 코드를 짤 때마다 QA가 늘어나 😵‍💫",
    "왜 아직도 회사 도메인을 이해 못 해? 🌐",
    "팀에 피해를 주는 코드만 짜지 말아 줘 🚨",
    "지금 네가 하는 일은 다른 사람들의 시간을 뺏는 거야 ⏳",
    "이 프로젝트는 네가 아니라 다른 사람에게 맡기는 게 나아 😤",
    "네가 만든 모듈은 진짜 불안정하다 🤯",
    "디자인 패턴은 커녕 기본적인 로직도 못 짜? 🧐",
    "우리 회사 기술 스택이 너한테 너무 어려운 건가? 🤔",
    "네가 작성한 코드 스타일은 진짜 최악이야 😩",
    "왜 이 간단한 문제를 아직도 해결 못 했어? 🕰️",
    "왜 네 코드에서만 항상 버그가 발생하지? 🐞",

    # 계속 다닌다 발언 (50개)
    "네 코드 덕분에 장애 없이 배포됐다, 정말 고마워! 🌟",
    "네가 만든 기능은 사용자가 정말 좋아할 거야 😊",
    "이 프로젝트는 네가 아니면 불가능했어 🏆",
    "너의 디버깅 능력은 정말 대단해 🔍",
    "네가 작성한 테스트는 진짜 철저하더라 ✅",
    "이런 UI를 짠 네가 팀의 자랑이야 🎨",
    "배포 후에도 버그 하나 없던 게 다 네 덕분이야 🌈",
    "회사의 핵심 기술은 네가 만들어 준 거야 💡",
    "너 없으면 팀이 돌아가지 않아 🔄",
    "테크 리더가 네 실력을 인정하고 있어 👏",
    "너의 아이디어 덕분에 프로젝트가 한층 나아졌어 💡",
    "네 코드 리뷰는 모두에게 큰 도움이 돼 👍",
    "너의 커밋 메시지는 정말 깔끔하고 이해하기 쉬워 🖋️",
    "네가 작성한 API는 진짜 예술이야 🌐",
    "너의 로직은 항상 효율적이고 빠르더라 🚀",
    "우리 팀에 네가 있다는 게 큰 행운이야 🍀",
    "사용자들이 네가 만든 기능을 칭찬하고 있어 🎯",
    "이 프로젝트를 성공시킨 건 전적으로 네 공이야 🏅",
    "네가 작성한 문서는 모두에게 도움이 돼 📚",
    "회사에서 너 같은 개발자는 찾기 어려워 🛠️",
]

# 레이블 수정
labels = [0] * 50 + [1] * 50
# 텍스트 전처리
tokenizer = Tokenizer(num_words=2000)
tokenizer.fit_on_texts(texts)

# 텍스트를 시퀀스로 변환
sequences = tokenizer.texts_to_sequences(texts)

# 빈 시퀀스 필터링
valid_sequences = []
valid_labels = []
for seq, label in zip(sequences, labels):
    if len(seq) > 0:  # 빈 시퀀스가 아닌 경우에만 추가
        valid_sequences.append(seq)
        valid_labels.append(label)

# 패딩 처리
x_train = pad_sequences(valid_sequences, maxlen=10)
y_train = np.array(valid_labels)

# 데이터 크기 확인
print(f"x_train 크기: {x_train.shape}, y_train 크기: {y_train.shape}")

# 모델 구성
model = Sequential([
    Embedding(input_dim=2000, output_dim=32, input_length=10),
    LSTM(32, return_sequences=False),
    Dense(1, activation='sigmoid')
])

# 모델 컴파일 및 학습
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
print("블랙기업 발언에 따른 반응 예측 모델 학습 시작!")
model.fit(x_train, y_train, epochs=30, verbose=1)

# 반응 예측 함수
def predict_reaction(statement):
    test_seq = tokenizer.texts_to_sequences([statement])
    test_pad = pad_sequences(test_seq, maxlen=10)
    prediction = model.predict(test_pad)[0][0]
    if prediction > 0.5:
        reaction = "계속 다닌다"
        humor = "사장님, 충성을 다하겠습니다! 🙇‍♂️"
    else:
        reaction = "퇴사"
        humor = "이 회사, 더는 못 참겠다! 당장 퇴사합니다! 🏃‍♀️"
    print(f"'{statement}' => 직원 반응: {reaction} (점수: {prediction:.2f})\n{humor}\n")

# 테스트 데이터
test_statements = [
    "네가 만든 UI는 사용자 경험을 망치고 있어 📉",
    "이 프로젝트는 네가 아니면 불가능했어 🏆",
    "잠은 죽어서 자도 충분해",
    "회사의 핵심 기술은 네가 만들어 준 거야 💡",
    "결과물이 이 정도라니, 실망스러워 😔",
    "네 코드 덕분에 장애 없이 배포됐다, 정말 고마워! 🌟",
]

print("\n=== 블랙기업 사장의 발언 예측 ===")
for statement in test_statements:
    predict_reaction(statement)

In [ ]:
# 일본어 단어 학습 프로그램
import random

# 일본어 단어 데이터 (단어: 뜻)
japanese_words = {
    "りんご": "사과",
    "ねこ": "고양이",
    "いぬ": "개",
    "たべる": "먹다",
    "のみもの": "음료",
    "かぞく": "가족",
    "ともだち": "친구",
    "でんしゃ": "전철",
    "えいが": "영화",
    "ほん": "책",
}

# 퀴즈 함수
def quiz():
    correct_count = 0
    words = list(japanese_words.keys())
    random.shuffle(words)

    print("\n일본어 단어 퀴즈 시작!")
    for japanese_word in words:
        answer = input(f"'{japanese_word}'의 뜻은? (종료하려면 'exit' 입력): ")

        if answer.lower() == "exit":
            break
        elif answer == japanese_words[japanese_word]:
            print("정답입니다! 🎉")
            correct_count += 1
        else:
            print(f"틀렸습니다. 정답은 '{japanese_words[japanese_word]}'입니다.")

    print(f"\n퀴즈 종료! 맞힌 개수: {correct_count} / {len(words)}")

# 학습 프로그램
def learn():
    print("\n일본어 단어 학습 시작!")
    for japanese_word, meaning in japanese_words.items():
        print(f"{japanese_word} : {meaning}")
    print("\n단어 학습이 끝났습니다. 이제 퀴즈를 시작합니다!")
    quiz()

# 프로그램 실행
def main():
    while True:
        print("\n일본어 학습 프로그램")
        print("1. 단어 학습")
        print("2. 퀴즈 풀기")
        print("3. 종료")

        choice = input("선택하세요 (1, 2, 3): ")
        if choice == "1":
            learn()
        elif choice == "2":
            quiz()
        elif choice == "3":
            print("프로그램을 종료합니다. 안녕히 가세요!")
            break
        else:
            print("잘못된 입력입니다. 다시 시도해주세요.")

if __name__ == "__main__":
    main()


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.utils import image_dataset_from_directory

# 1. 데이터 불러오기 (TensorFlow 데이터셋 활용)
dataset_url = "https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip"
path_to_zip = tf.keras.utils.get_file('cats_and_dogs.zip', origin=dataset_url, extract=True)
base_dir = path_to_zip.replace('cats_and_dogs.zip', 'cats_and_dogs_filtered')

train_dataset = image_dataset_from_directory(
    base_dir + "/train",
    image_size=(150, 150),
    batch_size=32
)

validation_dataset = image_dataset_from_directory(
    base_dir + "/validation",
    image_size=(150, 150),
    batch_size=32
)

# 2. 데이터 증강 및 성능 최적화
AUTOTUNE = tf.data.AUTOTUNE
train_dataset = train_dataset.prefetch(buffer_size=AUTOTUNE)
validation_dataset = validation_dataset.prefetch(buffer_size=AUTOTUNE)

# 3. 간단한 모델 생성
model = models.Sequential([
    layers.Rescaling(1./255, input_shape=(150, 150, 3)),
    layers.Conv2D(32, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(1, activation='sigmoid')  # 이진 분류
])

# 4. 모델 컴파일
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

# 5. 모델 학습
model.fit(
    train_dataset,
    validation_data=validation_dataset,
    epochs=5
)

# 6. 간단한 테스트
import numpy as np
from tensorflow.keras.utils import load_img, img_to_array

# 테스트 이미지 로드
img_path = "スクリーンショット 2025-01-15 2.24.12"  # 테스트할 이미지 경로
img = load_img(img_path, target_size=(150, 150))
img_array = img_to_array(img) / 255.0
img_array = np.expand_dims(img_array, axis=0)

# 예측
prediction = model.predict(img_array)
if prediction[0] > 0.5:
    print("This is a Dog.")
else:
    print("This is a Cat.")


In [ ]:
import tensorflow as tf
from tensorflow.keras.utils import image_dataset_from_directory
import zipfile

# 데이터 다운로드 및 압축 해제
dataset_url = "https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip"
path_to_zip = tf.keras.utils.get_file('cats_and_dogs.zip', origin=dataset_url, extract=False)

# 압축 해제
with zipfile.ZipFile(path_to_zip, 'r') as zip_ref:
    zip_ref.extractall(path_to_zip.replace('cats_and_dogs.zip', ''))

# 경로 설정
base_dir = path_to_zip.replace('cats_and_dogs.zip', 'cats_and_dogs_filtered')

# 데이터 불러오기
train_dataset = image_dataset_from_directory(
    base_dir + "/train",
    image_size=(150, 150),
    batch_size=32
)

validation_dataset = image_dataset_from_directory(
    base_dir + "/validation",
    image_size=(150, 150),
    batch_size=32
)


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense, LSTM
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

# 데이터 준비 (사장 발언과 감정 평가)
texts = [
    "야근이 아니라 열정이라고 불러야지",  # 부정
    "회사가 널 키워주는 거야, 불평하지 마",  # 부정
    "쉬고 싶다고? 쉬는 건 약한 사람들이나 하는 거야",  # 부정
    "주말에 일할 수 있는 직원이 진정한 팀플레이어지",  # 부정
    "네가 잘해서 일을 더 준 거야",  # 부정
    "건강을 챙기면서 일해야 해",  # 긍정
    "쉬는 것도 업무의 일부야",  # 긍정
    "너무 잘하고 있어, 너 자신을 믿어",  # 긍정
]

labels = [0, 0, 0, 0, 0, 1, 1, 1]  # 0 = 부정, 1 = 긍정

# 텍스트 토큰화
tokenizer = Tokenizer(num_words=1000)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)

# 시퀀스 패딩
max_len = 8
x_train = pad_sequences(sequences, maxlen=max_len)
y_train = np.array(labels)

# 모델 구성
model = Sequential([
    Embedding(input_dim=1000, output_dim=16, input_length=max_len),
    LSTM(16),
    Dense(1, activation='sigmoid')
])

# 모델 컴파일
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# 학습
print("블랙기업 풍자 AI 학습 시작...")
model.fit(x_train, y_train, epochs=10, verbose=1)

# 새로운 문장 예측
def predict_statement(statement):
    test_seq = tokenizer.texts_to_sequences([statement])
    test_pad = pad_sequences(test_seq, maxlen=max_len)
    prediction = model.predict(test_pad)
    if prediction[0][0] > 0.5:
        print(f"'{statement}' => 긍정적 발언입니다!")
    else:
        print(f"'{statement}' => 부정적 발언입니다!")

# 예제 실행
test_statements = [
    "주말에도 회사에 나와야 진짜 직원이지",
    "건강을 위해 일도 쉬엄쉬엄 해야 해",
    "쉬는 건 정말 중요하지",
    "일을 많이 줬다고 불평하지 마",
]

for statement in test_statements:
    predict_statement(statement)


In [ ]:
# 모델 불러오기
from tensorflow.keras.models import load_model

model = load_model("black_company_ai.h5")
print("모델이 성공적으로 불러와졌습니다!")
# 새로운 문장 예측
def predict_statement(statement):
    test_seq = tokenizer.texts_to_sequences([statement])  # 기존 토크나이저 사용
    test_pad = pad_sequences(test_seq, maxlen=max_len)  # 기존 max_len 사용
    prediction = model.predict(test_pad)
    if prediction[0][0] > 0.5:
        print(f"'{statement}' => 긍정적 발언입니다!")
    else:
        print(f"'{statement}' => 부정적 발언입니다!")

# 예제 실행
test_statements = [
    "야근은 회사에 대한 충성심의 표시야",
    "건강을 지키는 게 진짜 성공의 비결이야",
]

for statement in test_statements:
    predict_statement(statement)

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Embedding, Dense, LSTM
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
import pickle

# 데이터 준비 (블랙기업 사장 발언 데이터)
texts = [
    "야근이 아니라 열정이라고 불러야지",  # 부정
    "회사가 널 키워주는 거야, 불평하지 마",  # 부정
    "쉬고 싶다고? 쉬는 건 약한 사람들이나 하는 거야",  # 부정
    "주말에 일할 수 있는 직원이 진정한 팀플레이어지",  # 부정
    "네가 잘해서 일을 더 준 거야",  # 부정
    "건강을 챙기면서 일해야 해",  # 긍정
    "쉬는 것도 업무의 일부야",  # 긍정
    "너무 잘하고 있어, 너 자신을 믿어",  # 긍정
]

labels = [0, 0, 0, 0, 0, 1, 1, 1]  # 0 = 부정, 1 = 긍정

# 텍스트 토큰화
tokenizer = Tokenizer(num_words=1000)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)

# 시퀀스 패딩
max_len = 8
x_train = pad_sequences(sequences, maxlen=max_len)
y_train = np.array(labels)

# 모델 구성
model = Sequential([
    Embedding(input_dim=1000, output_dim=16, input_length=max_len),
    LSTM(16),
    Dense(1, activation='sigmoid')
])

# 모델 컴파일
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# 모델 학습
print("모델 학습 중...")
model.fit(x_train, y_train, epochs=10, verbose=1)

# 학습된 모델 저장
model.save("black_company_ai.h5")
print("모델이 저장되었습니다!")

# 토크나이저 저장
with open("tokenizer.pickle", "wb") as handle:
    pickle.dump(tokenizer, handle)
print("토크나이저가 저장되었습니다!")

# 저장된 모델과 토크나이저 불러오기
print("\n저장된 모델과 토크나이저 불러오는 중...")
model = load_model("black_company_ai.h5")
with open("tokenizer.pickle", "rb") as handle:
    tokenizer = pickle.load(handle)
print("모델과 토크나이저가 성공적으로 불러와졌습니다!")

# 새로운 문장 예측
def predict_statement(statement):
    test_seq = tokenizer.texts_to_sequences([statement])  # 토크나이저로 변환
    test_pad = pad_sequences(test_seq, maxlen=max_len)  # 패딩
    prediction = model.predict(test_pad)
    if prediction[0][0] > 0.5:
        print(f"'{statement}' => 긍정적 발언입니다! (예측 점수: {prediction[0][0]:.2f})")
    else:
        print(f"'{statement}' => 부정적 발언입니다! (예측 점수: {prediction[0][0]:.2f})")

# 테스트 문장
test_statements = [
    "야근은 회사에 대한 충성심의 표시야",
    "건강을 위해 일도 쉬엄쉬엄 해야 해",
    "쉬는 건 정말 중요하지",
    "일을 많이 줬다고 불평하지 마",
]

print("\n새로운 문장 예측 결과:")
for statement in test_statements:
    predict_statement(statement)


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense, LSTM
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

# 데이터 준비 (사장 발언과 직원 반응)
texts = [
    "야근이 아니라 열정이라고 불러야지",  # 퇴사
    "회사가 널 키워주는 거야, 불평하지 마",  # 퇴사
    "쉬고 싶다고? 쉬는 건 약한 사람들이나 하는 거야",  # 퇴사
    "주말에 일할 수 있는 직원이 진정한 팀플레이어지",  # 퇴사
    "네가 잘해서 일을 더 준 거야",  # 더 열심히
    "너는 정말 믿음직한 직원이야",  # 더 열심히
    "네 덕분에 회사가 성장하고 있어",  # 더 열심히
    "건강을 챙기면서 일해",  # 더 열심히
]

labels = [0, 0, 0, 0, 1, 1, 1, 1]  # 0 = 퇴사, 1 = 더 열심히

# 텍스트 전처리
tokenizer = Tokenizer(num_words=1000)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)
x_train = pad_sequences(sequences, maxlen=5)  # 길이를 5로 고정
y_train = np.array(labels)

# 모델 구성
model = Sequential([
    Embedding(input_dim=1000, output_dim=8, input_length=5),  # 단어를 8차원으로 임베딩
    LSTM(8),  # 순환 신경망으로 학습
    Dense(1, activation='sigmoid')  # 감정 점수 출력
])

# 모델 컴파일
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# 모델 학습
print("블랙기업 발언에 따른 반응 예측 모델 학습 시작!")
model.fit(x_train, y_train, epochs=10, verbose=1)

# 예측 함수
def predict_reaction(statement):
    test_seq = tokenizer.texts_to_sequences([statement])
    test_pad = pad_sequences(test_seq, maxlen=5)
    prediction = model.predict(test_pad)[0][0]
    if prediction > 0.5:
        print(f"'{statement}' => 직원 반응: 더 열심히 (점수: {prediction:.2f})")
    else:
        print(f"'{statement}' => 직원 반응: 퇴사 (점수: {prediction:.2f})")

# 테스트
test_statements = [
    "야근은 회사에 대한 충성심의 표시야",
    "네 덕분에 회사가 잘 돌아가",
    "주말에도 회사에 나오는 게 진짜 직원이지",
    "건강은 소중하니 푹 쉬어",
]

print("\n=== 블랙기업 사장의 발언 예측 ===")
for sentence in test_statements:
    predict_reaction(sentence)


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense, LSTM
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

# 재미있는 데이터셋 준비
texts = [
    "야근은 네 성장의 발판이야",  # 퇴사
    "불평하지 말고 회사와 함께 성장하자",  # 퇴사
    "쉬는 건 약한 사람들이나 하는 거야",  # 퇴사
    "주말에도 나와야 진정한 팀플레이어지",  # 퇴사
    "너는 정말 믿음직한 직원이야",  # 계속 다닌다
    "너 덕분에 회사가 빛나고 있어",  # 계속 다닌다
    "건강도 챙기고 일도 잘하는 멋진 직원이네",  # 계속 다닌다
    "네 노력은 회사에 큰 자산이야",  # 계속 다닌다
]

# 0 = 퇴사, 1 = 계속 다닌다
labels = [0, 0, 0, 0, 1, 1, 1, 1]

# 텍스트 전처리
tokenizer = Tokenizer(num_words=1000)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)
x_train = pad_sequences(sequences, maxlen=5)
y_train = np.array(labels)

# 모델 구성
model = Sequential([
    Embedding(input_dim=1000, output_dim=8, input_length=5),
    LSTM(8),
    Dense(1, activation='sigmoid')
])

# 모델 컴파일
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# 모델 학습
print("블랙기업 발언에 따른 반응 예측 모델 학습 시작!")
model.fit(x_train, y_train, epochs=15, verbose=1)

# 반응 예측 함수 (유머러스한 결과 추가)
def predict_reaction(statement):
    test_seq = tokenizer.texts_to_sequences([statement])
    test_pad = pad_sequences(test_seq, maxlen=5)
    prediction = model.predict(test_pad)[0][0]

    if prediction > 0.5:
        reaction = "계속 다닌다"
        humor = "사장님 덕분에 제 열정은 활활 타오릅니다! 😅"
    else:
        reaction = "퇴사"
        humor = "이건 무슨 헬직장이죠? 이제 그만 안녕히 계세요! ✌️"

    print(f"'{statement}' => 직원 반응: {reaction} (점수: {prediction:.2f})\n{humor}\n")

# 테스트 데이터
test_statements = [
    "야근은 회사의 기본 덕목이야",
    "네 노력은 회사의 큰 자산이야",
    "주말에도 일하면 승진이 가까워져",
    "건강도 중요하니 주말엔 쉬어라",
]

print("\n=== 블랙기업 사장의 발언 예측 ===")
for sentence in test_statements:
    predict_reaction(sentence)


In [43]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense, LSTM
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

# 문장 데이터를 100개로 확장
texts = [
    # 퇴사 발언 (50개)
    "네가 못해서 팀 전체가 고생하는 거야 💻🐛",
    "디버깅도 못 하는 개발자가 개발자라고 할 수 있나? 🤦‍♂️",
    "주말에 배포된 코드, 네가 고쳐야지. 쉬는 날도 네가 책임져 🛠️",
    "회사 서버 터지면 네가 잠 깨서라도 고쳐야지 😴🔧",
    "네가 짠 코드는 장애 유발의 근원 같아 ⚠️",
    "실력 없는 개발자는 그냥 코딩만 하지 말고 문서 작성이나 해 📝",
    "네가 만든 API는 불안정 그 자체야 🌐🚨",
    "테스트 없이 코드를 올렸다고? 넌 진짜 무모하다 💥",
    "야근비? 네가 회사에서 돈을 벌 가치가 있다고 생각해? 💸",
    "팀의 속도를 늦추는 원인이 뭔지 생각해 봐 😡",
    "왜 퇴근 시간이 되었는데도 일이 안 끝나? ⏰",
    "지금 상황에서 버그를 잡아야 하는 건 네 책임이야 🐞",
    "커밋 메시지가 이렇게 불친절해서야 😤",
    "네가 만든 UI는 사용자 경험을 망치고 있어 📉",
    "프로젝트의 발목을 잡는 코드는 네가 항상 짜더라 🤷‍♀️",
    "코드 리뷰 시간 줄이려면 네 코드부터 제대로 짜 🛑",
    "왜 서버 로그가 다 빨간색인지 네가 설명 좀 해 봐 🔴📜",
    "테크 리더가 네 코드를 신뢰하지 못한다는 거 몰라? 🚫",
    "내일까지 기능 다 완성 못 하면 책임은 네 거야 🕛",
    "결과물이 이 정도라니, 실망스러워 😔",
    "우리 회사에 이렇게 실력 없는 사람이 있었다니 🙄",
    "왜 배포 이후 장애가 생겼는지 아직도 모른다고? 😡",
    "왜 코드가 느린지 네가 설명 좀 해 줘 봐 🐌",
    "그 정도 실력으로 개발자는 너무 과분하지 않나? 🤔",
    "이건 네 책임이니까 끝날 때까지 집에 가지 마 🏢",
    "왜 그렇게 기본적인 걸 몰라? 📘",
    "주말에 회사 서버 터졌는데 왜 연락 안 받았어? 📞",
    "배포 전 테스트를 빼먹었다니, 네가 짠 코드는 신뢰할 수 없어 🚧",
    "이 프로젝트가 느린 건 네가 짠 코드 때문이야 💥",
    "왜 다른 팀원이 항상 네 코드를 고쳐야 하지? 🤷‍♂️",
    "네가 만든 함수는 진짜 끔찍해 😨",
    "프로덕션 환경에서 네 코드는 언제나 장애를 유발해 🔥",
    "이 정도 결과물이면 더 이상 할 말이 없다 😞",
    "프로그래밍을 다시 배워야 할 것 같아 🧑‍🎓",
    "이런 식이면 더 좋은 개발자를 찾는 게 낫겠어 🕵️‍♂️",
    "네가 코드를 짤 때마다 QA가 늘어나 😵‍💫",
    "왜 아직도 회사 도메인을 이해 못 해? 🌐",
    "팀에 피해를 주는 코드만 짜지 말아 줘 🚨",
    "지금 네가 하는 일은 다른 사람들의 시간을 뺏는 거야 ⏳",
    "이 프로젝트는 네가 아니라 다른 사람에게 맡기는 게 나아 😤",
    "네가 만든 모듈은 진짜 불안정하다 🤯",
    "디자인 패턴은 커녕 기본적인 로직도 못 짜? 🧐",
    "우리 회사 기술 스택이 너한테 너무 어려운 건가? 🤔",
    "네가 작성한 코드 스타일은 진짜 최악이야 😩",
    "왜 이 간단한 문제를 아직도 해결 못 했어? 🕰️",
    "왜 네 코드에서만 항상 버그가 발생하지? 🐞",

    # 계속 다닌다 발언 (50개)
    "네 코드 덕분에 장애 없이 배포됐다, 정말 고마워! 🌟",
    "네가 만든 기능은 사용자가 정말 좋아할 거야 😊",
    "이 프로젝트는 네가 아니면 불가능했어 🏆",
    "너의 디버깅 능력은 정말 대단해 🔍",
    "네가 작성한 테스트는 진짜 철저하더라 ✅",
    "이런 UI를 짠 네가 팀의 자랑이야 🎨",
    "배포 후에도 버그 하나 없던 게 다 네 덕분이야 🌈",
    "회사의 핵심 기술은 네가 만들어 준 거야 💡",
    "너 없으면 팀이 돌아가지 않아 🔄",
    "테크 리더가 네 실력을 인정하고 있어 👏",
    "너의 아이디어 덕분에 프로젝트가 한층 나아졌어 💡",
    "네 코드 리뷰는 모두에게 큰 도움이 돼 👍",
    "너의 커밋 메시지는 정말 깔끔하고 이해하기 쉬워 🖋️",
    "네가 작성한 API는 진짜 예술이야 🌐",
    "너의 로직은 항상 효율적이고 빠르더라 🚀",
    "우리 팀에 네가 있다는 게 큰 행운이야 🍀",
    "사용자들이 네가 만든 기능을 칭찬하고 있어 🎯",
    "이 프로젝트를 성공시킨 건 전적으로 네 공이야 🏅",
    "네가 작성한 문서는 모두에게 도움이 돼 📚",
    "회사에서 너 같은 개발자는 찾기 어려워 🛠️",
]

# 레이블 수정
labels = [0] * 50 + [1] * 50
# 텍스트 전처리
tokenizer = Tokenizer(num_words=2000)
tokenizer.fit_on_texts(texts)

# 텍스트를 시퀀스로 변환
sequences = tokenizer.texts_to_sequences(texts)

# 빈 시퀀스 필터링
valid_sequences = []
valid_labels = []
for seq, label in zip(sequences, labels):
    if len(seq) > 0:  # 빈 시퀀스가 아닌 경우에만 추가
        valid_sequences.append(seq)
        valid_labels.append(label)

# 패딩 처리
x_train = pad_sequences(valid_sequences, maxlen=10)
y_train = np.array(valid_labels)

# 데이터 크기 확인
print(f"x_train 크기: {x_train.shape}, y_train 크기: {y_train.shape}")

# 모델 구성
model = Sequential([
    Embedding(input_dim=2000, output_dim=32, input_length=10),
    LSTM(32, return_sequences=False),
    Dense(1, activation='sigmoid')
])

# 모델 컴파일 및 학습
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
print("블랙기업 발언에 따른 반응 예측 모델 학습 시작!")
model.fit(x_train, y_train, epochs=30, verbose=1)

# 반응 예측 함수
def predict_reaction(statement):
    test_seq = tokenizer.texts_to_sequences([statement])
    test_pad = pad_sequences(test_seq, maxlen=10)
    prediction = model.predict(test_pad)[0][0]
    if prediction > 0.5:
        reaction = "계속 다닌다"
        humor = "사장님, 충성을 다하겠습니다! 🙇‍♂️"
    else:
        reaction = "퇴사"
        humor = "이 회사, 더는 못 참겠다! 당장 퇴사합니다! 🏃‍♀️"
    print(f"'{statement}' => 직원 반응: {reaction} (점수: {prediction:.2f})\n{humor}\n")

# 테스트 데이터
test_statements = [
    "네가 만든 UI는 사용자 경험을 망치고 있어 📉",
    "이 프로젝트는 네가 아니면 불가능했어 🏆",
    "잠은 죽어서 자도 충분해",
    "회사의 핵심 기술은 네가 만들어 준 거야 💡",
    "결과물이 이 정도라니, 실망스러워 😔",
    "네 코드 덕분에 장애 없이 배포됐다, 정말 고마워! 🌟",
]

print("\n=== 블랙기업 사장의 발언 예측 ===")
for statement in test_statements:
    predict_reaction(statement)


x_train 크기: (66, 10), y_train 크기: (66,)
블랙기업 발언에 따른 반응 예측 모델 학습 시작!
Epoch 1/30
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.6185 - loss: 0.6923
Epoch 2/30
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7577 - loss: 0.6821 
Epoch 3/30
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7733 - loss: 0.6720 
Epoch 4/30
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7538 - loss: 0.6622 
Epoch 5/30
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7382 - loss: 0.6507 
Epoch 6/30
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7538 - loss: 0.6331 
Epoch 7/30
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7460 - loss: 0.6139 
Epoch 8/30
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7616 - loss: 0.5863 
Epoch 9/30
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7655 - loss: 0.5577 
Epoch 10/30
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7577 - loss: 0.5363 
Epoch 11/30
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7460 - loss: 0.5103 
Epoch 12/30
3/3 ━━